In [1]:
!pip uninstall -y gymnasium gym
!pip install "gymnasium[atari]==0.29.0" --quiet
!pip install "stable-baselines3[extra]==2.1.0" --quiet
!pip install autorom[accept-rom-license] --quiet

# Descargar ROMs de Atari
!AutoROM --accept-license


Found existing installation: gymnasium 0.29.0
Uninstalling gymnasium-0.29.0:
  Successfully uninstalled gymnasium-0.29.0
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 13.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.1 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, which is not installed.
kaggle-environments 1.18.0 requires shimmy>=1.2.1, but you have shimmy 0.2.1 which is incompatible.
dopamine-rl 4.1.2 requires ale-py>=0.10.1, but you have ale-py 0.8.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.7 MB/s eta 0

In [2]:
import gymnasium as gym

env = gym.make("ALE/Galaxian-v5", render_mode="rgb_array")

obs, info = env.reset()
print("Obs:", obs.shape)
print("Actions:", env.action_space)


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Obs: (210, 160, 3)
Actions: Discrete(6)


In [3]:
!pip uninstall -y tensorflow tensorboard tensorboardX tb-nightly stable-baselines3


Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorboard 2.18.0
Uninstalling tensorboard-2.18.0:
  Successfully uninstalled tensorboard-2.18.0
Found existing installation: stable-baselines3 2.1.0
Uninstalling stable-baselines3-2.1.0:
  Successfully uninstalled stable-baselines3-2.1.0


In [4]:
!pip install stable-baselines3==2.1.0 --no-deps
!pip install torch torchvision --quiet
!pip install gymnasium[atari]==0.29.0 --quiet
!pip install autorom[accept-rom-license] --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.4 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.18.0 requires shimmy>=1.2.1, but you have shimmy 0.2.1 which is incompatible.


In [5]:
!pip install numpy cloudpickle opencv-python


In [6]:
from stable_baselines3 import DQN

model = DQN(
    "CnnPolicy",
    env,
    learning_rate=1e-4,
    buffer_size=200000,          # más memoria = mejor para juegos largos
    learning_starts=50000,       # el original usa 50k
    batch_size=32,
    train_freq=4,
    target_update_interval=10000,
    exploration_fraction=0.3,    # ← muy importante
    exploration_final_eps=0.01,
        tensorboard_log=None,   # 🔥 desactivar tensorboard
    verbose=1
)

model.learn(total_timesteps=150_000)
model.save("dqn_galaxian")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 40.32GB > 31.90GB
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 544      |
|    ep_rew_mean      | 750      |
|    exploration_rate | 0.952    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 897      |
|    time_elapsed     | 2        |
|    total_timesteps  | 2178     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 658      |
|    ep_rew_mean      | 780      |
|    exploration_rate | 0.884    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 918      |
|    time_elapsed     | 5        |
|    total_timesteps  | 5266     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 600      |
|    ep_rew_mean      | 690      |
|    exploration_rate | 0.842    |
| time/               |          |
|    episodes       

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)
print("Reward promedio:", mean_reward)


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Reward promedio: 464.0


In [8]:
import imageio
import numpy as np

def generar_video(model, filename="galaxian_agent.mp4"):
    env = gym.make("ALE/Galaxian-v5", render_mode="rgb_array")
    obs, info = env.reset()
    frames = []
    done = False

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        frames.append(env.render())
        done = terminated or truncated

    imageio.mimsave(filename, frames, fps=30)
    print("Video guardado:", filename)

generar_video(model)


/usr/local/lib/python3.11/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Video guardado: galaxian_agent.mp4


In [9]:
from IPython.display import Video
Video("galaxian_agent.mp4", embed=True)
